In [1]:
import sys
sys.path.append("/home/ubuntu/github/mkardas/paper-extractor")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sota_extractor2.data.paper_collection import PaperCollection
from pathlib import Path

#DATA_PATH = Path("/home/ubuntu/pwc/arxiv-s3/arxiv")
DATA_PATH = Path("/home/ubuntu/pwc/arxiv-pwc/arxiv")

pc = PaperCollection(DATA_PATH)

In [4]:
PICKLE_PATH = Path("/home/ubuntu/pwc/pc-pickle-fast.pkl")
pc.to_pickle(PICKLE_PATH)

In [5]:
%time pc2 = PaperCollection.from_pickle(PICKLE_PATH)

CPU times: user 2.78 s, sys: 149 ms, total: 2.93 s
Wall time: 2.9 s


In [6]:
paper = pc.papers['1607.04315']
table = paper.tables[0]
table.display()

Model,d,|Î¸|M,Train,Test
Classifier with handcrafted features [12],-,-,99.7,78.2
LSTM encoders [12],300,3.0M,83.9,80.6
Dependency Tree CNN encoders [13],300,3.5M,83.3,82.1
SPINN-PI encoders [14],300,3.7M,89.2,83.2
NSE,300,3.4M,86.2,84.6
MMA-NSE,300,6.3M,87.1,84.8
LSTM attention [15],100,242K,85.4,82.3
LSTM word-by-word attention [15],100,252K,85.3,83.5
MMA-NSE attention,300,6.5M,86.9,85.4
mLSTM word-by-word attention [16],300,1.9M,92.0,86.1


In [7]:
table.df.iloc[1,0]

Cell(value='Classifier with handcrafted features [12]', gold_tags='model-competing', refs=['xxref-Xbowman:15'])

In [8]:
table.gold_tags

'sota'

In [9]:
paper.text.abstract

'Abstract We present a memory augmented neural network for natural language understanding: Neural Semantic Encoders. NSE is equipped with a novel memory update rule and has a variable sized encoding memory that evolves over time and maintains the understanding of input sequences through read , compose and write operations. NSE can also access 1 xxanchor-x1-2f1 multiple and shared memories. In this paper, we demonstrated the effectiveness and the flexibility of NSE on five different natural language tasks: natural language inference, question answering, sentence classification, document sentiment analysis and machine translation where NSE achieved state-of-the-art performance when evaluated on publically available benchmarks. For example, our shared-memory model showed an encouraging result on neural machine translation, improving an attention-based baseline by approximately 1.0 BLEU.'

In [10]:
paper.text.print_toc()

1 xxanchor-x1-10001 Introduction

2 xxanchor-x1-20002 Related Work

3 xxanchor-x1-30003 Proposed Approach

3.1 xxanchor-x1-40003.1 Read, Compose and Write

3.2 xxanchor-x1-50003.2 Shared and Multiple Memory Accesses

...

5.1 xxanchor-x1-130005.1 Memory Access and Compositionality

6 xxanchor-x1-140006 Conclusion

xxanchor-x1-150006 Acknowledgments

xxanchor-x1-160006 References

A xxanchor-x1-17000A Step-by-step visualization of memory states in NSE

In [11]:
paper.text.fragments[1]

# 1 xxanchor-x1-10001 Introduction,
Recently several studies have explored ways of extending the neural networks with an external memory [ xxref-Xgraves2014neural – xxref-Xgrefenstette2015learning ]. Unlike LSTM, the short term memories and the training parameters of such a neural network are no longer coupled and can be adapted. In this paper we propose a novel class of memory augmented neural networks called Neural Semantic Encoders (NSE) for natural language understanding. NSE offers several desirable properties. NSE has a variable sized encoding memory which allows the model to access entire input sequence during the reading process; therefore efficiently delivering long-term dependencies over time. The encoding memory evolves over time and maintains the memory of the input sequence through read , compose and write operations. NSE sequentially processes the input and supports word compositionality inheriting both temporal and hierarchical nature of human language. NSE can read from